In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import Data_for_bp as dfb

for the 1st very simple strategy lets try day Aug 15 2023 to Aug 17 trade

In [ ]:
tick_data = dfb.get_tick_data('15 Aug 2023','17 Aug 2023')['EURUSD.mid']

Let's try simple optimization problem when n (order size) is constant and equal 100000
Then
Objective function:
minimize {-Profit at D}

decision variables:
G = grid size
Position = position at time t # how about matrix
PNL@t = net profit and loss at time t # how about matrix
Profit@d = net profit and loss until day d # equal to PNL T (that day) + previous net profit day d-1 
parameter(known):
T = end of one trading day (1 day)
D = end of consideration period # Here will be Aug 16
d = day d

For this very first simple optimization lets use default scipy minimization method bfgs

In [ ]:
n=1000 # assume constant and equal to 100000
D=2 # Aug 15-Aug 16
# Group data by date
grouped = tick_data.groupby(tick_data.index.date)
T=np.zeros(D)
T_index = 0
for date, group in grouped:
    T[T_index] = len(group)
    print(f"Date: {date}, Number of ticks: {len(group)}")
    T_index += 1
T = T.astype(int)

In [ ]:
# lets reference price be first price when entering trade in that day
reference_price = np.zeros(D)
i=0
for j in range(D):
    reference_price[j] = tick_data[i]
    i += T[j-1]
reference_price

In [ ]:
def ladderize_open(tick_data, grid_size):
    """
    Convert tick data into step-based data using a specified grid size.

    :param tick_data: A pandas Series of tick data.
    :param grid_size: The size of the grid to discretize the tick data.
    :return: A pandas Series of ladderized data.
    """
    ladderized_data = [tick_data.iloc[0]]
    for i in range(1, len(tick_data)):
        if tick_data.iloc[i] > ladderized_data[-1] + grid_size:
            ladderized_data.append(ladderized_data[-1] + grid_size)
        elif tick_data.iloc[i] < ladderized_data[-1] - grid_size:
            ladderized_data.append(ladderized_data[-1] - grid_size)
        else:
            ladderized_data.append(ladderized_data[-1])
    # Adding the final close price
    ladderized_data[-1]=tick_data.iloc[-1]
    return pd.Series(ladderized_data, index=tick_data.index)
def ladderize_absolute(tick_data, grid_size):
    """
    Convert tick data into step-based data using a specified grid size.

    :param tick_data: A pandas Series of tick data.
    :param grid_size: The size of the grid to discretize the tick data.
    :return: A pandas Series of ladderized data.
    """
    # Initialize ladder at the nearest rounded price level based on grid size
    ladderized_data = [(tick_data.iloc[0] / grid_size).round() * grid_size]
    for i in range(1, len(tick_data)):
        current_tick = tick_data.iloc[i]
        last_ladder_level = ladderized_data[-1]
        if current_tick > last_ladder_level + grid_size:
            ladderized_data.append(last_ladder_level + grid_size)
        elif current_tick < last_ladder_level - grid_size:
            ladderized_data.append(last_ladder_level - grid_size)
        else:
            ladderized_data.append(last_ladder_level)
    # Adding the final close price
    ladderized_data[-1] = tick_data.iloc[-1]
    return pd.Series(ladderized_data, index=tick_data.index)

# Example usage:
# ladderized_data = ladderize_absolute(data, grid_size=0.0005)


In [ ]:
def compute_pnl_t(G,previous_position):
    #initialize pnl matrix for day d, T[d] period
    B = dfb.convert_to_binomial(tick_data, G, ladderize_open)
    pnl = np.zeros(T[d])
    #load position
    position = previous_position
    for t in range (T[d]-1):
        if B[t] > 0:
            position-=n
        elif B[t] < 0:
            position+=n
        else:
            position=position
        pnl[t+1] = position * G*B[t] + pnl[t] # should I change to pnl[t] += position * G*B[t] & t in range(T[d])
    pnl_T= pnl[-1] + position*(tick_data[T[d]-1]-G)
    previous_position = position
    return pnl,pnl_T,position,previous_position

In [ ]:
def net_profit(G):
    previous_position=0
    profit = np.zeros(D)
    for d in range (D):
        pnl,pnl_T,position = compute_pnl_t(G,previous_position)
        previous_position = position
        if d==0:
            profit[d] = pnl_T
        else:
            profit[d] = pnl_T + profit[d-1]
    #profit D
    net_profit = profit[-1] 
    return -net_profit

In [ ]:
# position should be between -10M and 10M
def position_constraint(params):
    G = params[0]
    _, pnl_T, position,previous_position = compute_pnl_t(G, previous_position)  
    return min(10000000 - abs(position), abs(position) + 10000000)  
# pnl should be greater than -150K
def pnl_constraint(params):
    G = params[0]
    pnl, _, _,previous_position = compute_pnl_t(G, previous_position)  
    return min(pnl) + 150000  
# profit should be greater than -500K
def profit_constraint(params):
    G = params[0]
    net_profit_value = net_profit(G)
    return net_profit_value + 500000  

In [ ]:
from scipy.optimize import minimize

# Create a list of constraints
constraints = [{'type': 'ineq', 'fun': position_constraint},
               {'type': 'ineq', 'fun': pnl_constraint},
               {'type': 'ineq', 'fun': profit_constraint}]

# Initial guess for G
initial_G = [0.1]

# Run the optimizer
result = minimize(lambda x: -net_profit(x[0]), 
                  initial_G,
                  constraints=constraints,
                  bounds=[(0, None)])  # G >= 0

# The optimal G should now be stored in result.x
optimal_G = result.x[0]
print(optimal_G)

In [ ]:
B=dfb.convert_to_binomial(tick_data,0.001,ladderize_absolute)

In [ ]:
grouped = B.groupby(B.index.date)
IB=np.zeros(D)
B_index = 0
for date, group in grouped:
    IB[B_index] = len(group)
    B_index += 1

In [ ]:
len(B)

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import mplfinance as mpf
import Data_for_bp as dfb
import pickle

tick_data = dfb.get_tick_data('15 Aug 2023','17 Aug 2023')['EURUSD.mid']
n = 100000
D = 2

grouped = tick_data.groupby(tick_data.index.date)
T=np.zeros(D)
T_index = 0
for date, group in grouped:
    T[T_index] = len(group)
    print(f"Date: {date}, Number of ticks: {len(group)}")
    T_index += 1
T = T.astype(int)
# lets reference price be first price when entering trade in that day
reference_price = np.zeros(D)
i=0
for j in range(D):
    reference_price[j] = tick_data[i]
    i += T[j-1]
reference_price
def ladderize_open(tick_data, grid_size):
    """
    Convert tick data into step-based data using a specified grid size.

    :param tick_data: A pandas Series of tick data.
    :param grid_size: The size of the grid to discretize the tick data.
    :return: A pandas Series of ladderized data.
    """
    ladderized_data = [tick_data.iloc[0]]
    for i in range(1, len(tick_data)):
        if tick_data.iloc[i] > ladderized_data[-1] + grid_size:
            ladderized_data.append(ladderized_data[-1] + grid_size)
        elif tick_data.iloc[i] < ladderized_data[-1] - grid_size:
            ladderized_data.append(ladderized_data[-1] - grid_size)
        else:
            ladderized_data.append(ladderized_data[-1])
    # Adding the final close price
    ladderized_data[-1]=tick_data.iloc[-1]
    return pd.Series(ladderized_data, index=tick_data.index)
def ladderize_absolute(tick_data, grid_size):
    """
    Convert tick data into step-based data using a specified grid size.

    :param tick_data: A pandas Series of tick data.
    :param grid_size: The size of the grid to discretize the tick data.
    :return: A pandas Series of ladderized data.
    """
    # Initialize ladder at the nearest rounded price level based on grid size
    ladderized_data = [(tick_data.iloc[0] / grid_size).round() * grid_size]
    for i in range(1, len(tick_data)):
        current_tick = tick_data.iloc[i]
        last_ladder_level = ladderized_data[-1]
        if current_tick > last_ladder_level + grid_size:
            ladderized_data.append(last_ladder_level + grid_size)
        elif current_tick < last_ladder_level - grid_size:
            ladderized_data.append(last_ladder_level - grid_size)
        else:
            ladderized_data.append(last_ladder_level)
    # Adding the final close price
    ladderized_data[-1] = tick_data.iloc[-1]
    return pd.Series(ladderized_data, index=tick_data.index)

def compute_pnl_t(tick_data, G, previous_position, T_day):
    B = dfb.convert_to_binomial1(tick_data, G, ladderize_open)
    pnl = np.zeros(T_day)
    position = previous_position
    
    for t in range(len(B)-1):
        if B[t] > 0:
            position -= n
        elif B[t] < 0:
            position += n
        else:
            position = position
        
        pnl[t + 1] = position * G * B[t] + pnl[t]
    
    pnl_T = pnl[-1] + position * (tick_data.iloc[len(B)-1] - G)
    return pnl, pnl_T, position

def net_profit(tick_data, G, D, T):
    previous_position = 0
    profit = np.zeros(D)
    start_idx = 0
    
    for d in range(D):
        T_day = T[d]
        end_idx = start_idx + T_day
        day_data = tick_data.iloc[start_idx:end_idx]
        
        pnl, pnl_T, position = compute_pnl_t(day_data, G, previous_position, T_day)
        previous_position = position
        
        if d == 0:
            profit[d] = pnl_T
        else:
            profit[d] = pnl_T + profit[d-1]
        
        start_idx = end_idx
    
    return -profit[-1]

def position_constraint(params, tick_data, D, T):
    G = params[0]
    _, _, position = compute_pnl_t(tick_data, G, 0, T[-1])
    return min(10000000 - abs(position), abs(position) + 10000000)

def pnl_constraint(params, tick_data, D, T):
    G = params[0]
    pnl, _, _ = compute_pnl_t(tick_data, G, 0, T[-1])
    return min(pnl) + 150000

def profit_constraint(params, tick_data, D, T):
    G = params[0]
    net_profit_value = net_profit(tick_data, G, D, T)
    return net_profit_value + 500000

# Create a list of constraints
constraints = [
    {'type': 'ineq', 'fun': lambda params: position_constraint(params, tick_data, D, T)},
    {'type': 'ineq', 'fun': lambda params: pnl_constraint(params, tick_data, D, T)},
    {'type': 'ineq', 'fun': lambda params: profit_constraint(params, tick_data, D, T)}
]

# Initial guess for G
initial_G = [0.0001]

# Run the optimizer
result = minimize(lambda x: -net_profit(tick_data, x[0], D, T), 
                  initial_G, 
                  constraints=constraints,
                  bounds=[(0, 1)])

# The optimal G should now be stored in result.x
optimal_G = result.x[0]
print(f"Optimal G: {optimal_G}")


In [ ]:
tick_data = dfb.get_tick_data('15 Aug 2023','16 Aug 2023')['EURUSD.mid']
tick_data

In [12]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import mplfinance as mpf
import Data_for_bp as dfb
import pickle

n = 100000

tick_data = dfb.get_tick_data('15 Aug 2023','16 Aug 2023')['EURUSD.mid']

# This will now only be one day
D = 1
T = np.array([len(tick_data)], dtype=int)

# Initial guess for G
initial_G = [0.0001]

# Single-day net_profit function
def net_profit_single_day(tick_data, G):
    pnl, _, _ = compute_pnl_t(tick_data, G, 0, T[0])
    return -pnl[-1]

# Single-day constraints
constraints = [
    {'type': 'ineq', 'fun': lambda params: position_constraint(params, tick_data, D, T)},
    {'type': 'ineq', 'fun': lambda params: pnl_constraint(params, tick_data, D, T)},
    {'type': 'ineq', 'fun': lambda params: -net_profit_single_day(tick_data, params[0]) + 500000}
]

# Run the optimizer for a single day
result = minimize(lambda x: net_profit_single_day(tick_data, x[0]),
                  initial_G,
                  constraints=constraints,
                  bounds=[(0, 1)])

# The optimal G for the single day should now be stored in result.x
optimal_G = result.x[0]
print(f"Optimal G for the single day: {optimal_G}")

def convert_to_binomial1(tick_data,grid_size,ladderized_function):
    '''
    '''
    ladderized_data = ladderized_function(tick_data,grid_size)
    jumps = filter_jumps(ladderized_data)
    aggregated_diff = aggregate_differences(jumps)
    binomial_data = aggregated_diff.diff()
    binomial_data.fillna(0, inplace=True)
    return binomial_data.to_list()# lets reference price be first price when entering trade in that day

def ladderize_open(tick_data, grid_size):
    """
    Convert tick data into step-based data using a specified grid size.

    :param tick_data: A pandas Series of tick data.
    :param grid_size: The size of the grid to discretize the tick data.
    :return: A pandas Series of ladderized data.
    """
    ladderized_data = [tick_data.iloc[0]]
    for i in range(1, len(tick_data)):
        if tick_data.iloc[i] > ladderized_data[-1] + grid_size:
            ladderized_data.append(ladderized_data[-1] + grid_size)
        elif tick_data.iloc[i] < ladderized_data[-1] - grid_size:
            ladderized_data.append(ladderized_data[-1] - grid_size)
        else:
            ladderized_data.append(ladderized_data[-1])
    # Adding the final close price
    ladderized_data[-1]=tick_data.iloc[-1]
    return pd.Series(ladderized_data, index=tick_data.index)
def ladderize_absolute(tick_data, grid_size):
    """
    Convert tick data into step-based data using a specified grid size.

    :param tick_data: A pandas Series of tick data.
    :param grid_size: The size of the grid to discretize the tick data.
    :return: A pandas Series of ladderized data.
    """
    # Initialize ladder at the nearest rounded price level based on grid size
    ladderized_data = [(tick_data.iloc[0] / grid_size).round() * grid_size]
    for i in range(1, len(tick_data)):
        current_tick = tick_data.iloc[i]
        last_ladder_level = ladderized_data[-1]
        if current_tick > last_ladder_level + grid_size:
            ladderized_data.append(last_ladder_level + grid_size)
        elif current_tick < last_ladder_level - grid_size:
            ladderized_data.append(last_ladder_level - grid_size)
        else:
            ladderized_data.append(last_ladder_level)
    # Adding the final close price
    ladderized_data[-1] = tick_data.iloc[-1]
    return pd.Series(ladderized_data, index=tick_data.index)

def compute_pnl_t(tick_data, G, previous_position, T_day):
    B = convert_to_binomial1(tick_data, G, ladderize_open)
    pnl = np.zeros(T_day)
    position = previous_position
    
    for t in range(T_day-1):
        if B[t] > 0:
            position -= n
        elif B[t] < 0:
            position += n
        else:
            position = position
        
        pnl[t + 1] = position * G * B[t] + pnl[t]
    
    pnl_T = pnl[-1] + position * (tick_data.iloc[T_day-1] - G)
    return pnl, pnl_T, position

def net_profit(tick_data, G, D, T):
    previous_position = 0
    profit = np.zeros(D)
    start_idx = 0
    
    for d in range(D):
        T_day = T[d]
        end_idx = start_idx + T_day
        day_data = tick_data.iloc[start_idx:end_idx]
        
        pnl, pnl_T, position = compute_pnl_t(day_data, G, previous_position, T_day)
        previous_position = position
        
        if d == 0:
            profit[d] = pnl_T
        else:
            profit[d] = pnl_T + profit[d-1]
        
        start_idx = end_idx
    
    return -profit[-1]

def position_constraint(params, tick_data, D, T):
    G = params[0]
    _, _, position = compute_pnl_t(tick_data, G, 0, T[-1])
    return min(10000000 - abs(position), abs(position) + 10000000)

def pnl_constraint(params, tick_data, D, T):
    G = params[0]
    pnl, _, _ = compute_pnl_t(tick_data, G, 0, T[-1])
    return min(pnl) + 150000

def profit_constraint(params, tick_data, D, T):
    G = params[0]
    net_profit_value = net_profit(tick_data, G, D, T)
    return net_profit_value + 500000

# Create a list of constraints
constraints = [
    {'type': 'ineq', 'fun': lambda params: position_constraint(params, tick_data, D, T)},
    {'type': 'ineq', 'fun': lambda params: pnl_constraint(params, tick_data, D, T)},
    {'type': 'ineq', 'fun': lambda params: profit_constraint(params, tick_data, D, T)}
]

# Initial guess for G
initial_G = [0.1]

# Run the optimizer
result = minimize(lambda x: -net_profit(tick_data, x[0], D, T), 
                  initial_G, 
                  constraints=constraints,
                  bounds=[(0, None)])

# The optimal G should now be stored in result.x
optimal_G = result.x[0]
print(f"Optimal G: {optimal_G}")


2023-09-16 14:58:01,028 - findatapy.market.datavendorweb - INFO - Request Dukascopy data
2023-09-16 14:58:01,029 - findatapy.market.datavendorweb - INFO - About to download from Dukascopy... for EURUSD
2023-09-16 14:58:01,034 - findatapy.market.datavendorweb - INFO - Downloading... 2023-08-15 00:00:00 https://www.dukascopy.com/datafeed/EURUSD/2023/07/15/00h_ticks.bi5


2023-09-16 14:58:06,426 - findatapy.market.datavendorweb - INFO - Completed request from Dukascopy
2023-09-16 14:58:08,504 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_858__abstract_curve_key-None__base_depos_currencies-EUR_GBP_AUD_NZD_USD_CAD_CHF_NOK_SEK_JPY__base_depos_tenor-ON_TN_SN_1W_2W_3W_1M_... to Redis: Error 10061 connecting to 127.0.0.1:6379. No connection could be made because the target machine actively refused it.


AttributeError: module 'Data_for_bp' has no attribute 'convert_to_binomial'

In [14]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import mplfinance as mpf
import Data_for_bp as dfb
import pickle

n = 100000

tick_data = dfb.get_tick_data('15 Aug 2023','16 Aug 2023')['EURUSD.mid']

# This will now only be one day
D = 1
T = np.array([len(tick_data)], dtype=int)

# Initial guess for G
initial_G = [0.0001]

# Single-day net_profit function
def net_profit_single_day(tick_data, G):
    pnl, _, _ = compute_pnl_t(tick_data, G, 0, T[0])
    return -pnl[-1]

# Single-day constraints
constraints = [
    {'type': 'ineq', 'fun': lambda params: position_constraint(params, tick_data, D, T)},
    {'type': 'ineq', 'fun': lambda params: pnl_constraint(params, tick_data, D, T)},
    {'type': 'ineq', 'fun': lambda params: -net_profit_single_day(tick_data, params[0]) + 500000}
]

# Run the optimizer for a single day
result = minimize(lambda x: net_profit_single_day(tick_data, x[0]),
                  initial_G,
                  constraints=constraints,
                  bounds=[(0, 1)])

# The optimal G for the single day should now be stored in result.x
optimal_G = result.x[0]
print(f"Optimal G for the single day: {optimal_G}")

def convert_to_binomial1(tick_data,grid_size,ladderized_function):
    '''
    '''
    ladderized_data = ladderized_function(tick_data,grid_size)
    jumps = filter_jumps(ladderized_data)
    aggregated_diff = aggregate_differences(jumps)
    binomial_data = aggregated_diff.diff()
    binomial_data.fillna(0, inplace=True)
    return binomial_data.to_list()# lets reference price be first price when entering trade in that day

def ladderize_open(tick_data, grid_size):
    """
    Convert tick data into step-based data using a specified grid size.

    :param tick_data: A pandas Series of tick data.
    :param grid_size: The size of the grid to discretize the tick data.
    :return: A pandas Series of ladderized data.
    """
    ladderized_data = [tick_data.iloc[0]]
    for i in range(1, len(tick_data)):
        if tick_data.iloc[i] > ladderized_data[-1] + grid_size:
            ladderized_data.append(ladderized_data[-1] + grid_size)
        elif tick_data.iloc[i] < ladderized_data[-1] - grid_size:
            ladderized_data.append(ladderized_data[-1] - grid_size)
        else:
            ladderized_data.append(ladderized_data[-1])
    # Adding the final close price
    ladderized_data[-1]=tick_data.iloc[-1]
    return pd.Series(ladderized_data, index=tick_data.index)
def ladderize_absolute(tick_data, grid_size):
    """
    Convert tick data into step-based data using a specified grid size.

    :param tick_data: A pandas Series of tick data.
    :param grid_size: The size of the grid to discretize the tick data.
    :return: A pandas Series of ladderized data.
    """
    # Initialize ladder at the nearest rounded price level based on grid size
    ladderized_data = [(tick_data.iloc[0] / grid_size).round() * grid_size]
    for i in range(1, len(tick_data)):
        current_tick = tick_data.iloc[i]
        last_ladder_level = ladderized_data[-1]
        if current_tick > last_ladder_level + grid_size:
            ladderized_data.append(last_ladder_level + grid_size)
        elif current_tick < last_ladder_level - grid_size:
            ladderized_data.append(last_ladder_level - grid_size)
        else:
            ladderized_data.append(last_ladder_level)
    # Adding the final close price
    ladderized_data[-1] = tick_data.iloc[-1]
    return pd.Series(ladderized_data, index=tick_data.index)

def compute_pnl_t(tick_data, G, previous_position, T_day):
    B = dfb.convert_to_binomial(tick_data, G, ladderize_open)
    pnl = np.zeros(T_day)
    position = previous_position
    
    for t in range(T_day-1):
        if B[t] > 0:
            position -= n
        elif B[t] < 0:
            position += n
        else:
            position = position
        
        pnl[t + 1] = position * G * B[t] + pnl[t]
    
    pnl_T = pnl[-1] + position * (tick_data.iloc[T_day-1] - G)
    return pnl, pnl_T, position

def net_profit(tick_data, G, D, T):
    previous_position = 0
    profit = np.zeros(D)
    start_idx = 0
    
    for d in range(D):
        T_day = T[d]
        end_idx = start_idx + T_day
        day_data = tick_data.iloc[start_idx:end_idx]
        
        pnl, pnl_T, position = compute_pnl_t(day_data, G, previous_position, T_day)
        previous_position = position
        
        if d == 0:
            profit[d] = pnl_T
        else:
            profit[d] = pnl_T + profit[d-1]
        
        start_idx = end_idx
    
    return -profit[-1]

def position_constraint(params, tick_data, D, T):
    G = params[0]
    _, _, position = compute_pnl_t(tick_data, G, 0, T[-1])
    return min(10000000 - abs(position), abs(position) + 10000000)

def pnl_constraint(params, tick_data, D, T):
    G = params[0]
    pnl, _, _ = compute_pnl_t(tick_data, G, 0, T[-1])
    return min(pnl) + 150000

def profit_constraint(params, tick_data, D, T):
    G = params[0]
    net_profit_value = net_profit(tick_data, G, D, T)
    return net_profit_value + 500000

# Create a list of constraints
constraints = [
    {'type': 'ineq', 'fun': lambda params: position_constraint(params, tick_data, D, T)},
    {'type': 'ineq', 'fun': lambda params: pnl_constraint(params, tick_data, D, T)},
    {'type': 'ineq', 'fun': lambda params: profit_constraint(params, tick_data, D, T)}
]

# Initial guess for G
initial_G = [0.1]

# Run the optimizer
result = minimize(lambda x: -net_profit(tick_data, x[0], D, T), 
                  initial_G, 
                  constraints=constraints,
                  bounds=[(0, None)])

# The optimal G should now be stored in result.x
optimal_G = result.x[0]
print(f"Optimal G: {optimal_G}")


2023-09-16 14:59:53,034 - findatapy.market.datavendorweb - INFO - Request Dukascopy data
2023-09-16 14:59:53,034 - findatapy.market.datavendorweb - INFO - About to download from Dukascopy... for EURUSD
2023-09-16 14:59:53,039 - findatapy.market.datavendorweb - INFO - Downloading... 2023-08-15 00:00:00 https://www.dukascopy.com/datafeed/EURUSD/2023/07/15/00h_ticks.bi5


2023-09-16 14:59:58,377 - findatapy.market.datavendorweb - INFO - Completed request from Dukascopy
2023-09-16 15:00:00,442 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_858__abstract_curve_key-None__base_depos_currencies-EUR_GBP_AUD_NZD_USD_CAD_CHF_NOK_SEK_JPY__base_depos_tenor-ON_TN_SN_1W_2W_3W_1M_... to Redis: Error 10061 connecting to 127.0.0.1:6379. No connection could be made because the target machine actively refused it.


AttributeError: module 'Data_for_bp' has no attribute 'convert_to_binomial'